In [6]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [145]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

# Gradient Descent Algorithm

We can think of a function of two or more variables. We have an objective function e.g. $$\large L(\beta_1, \beta_2)$$

We hope to share some ideas when $$L(\vec{\beta})$$ is convex in the vector $\vec{\beta}$

In particular we are interested in the situation when $L$ is the regularized sum of squarred errors, such as:

$$L: = \frac{1}{n}\displaystyle\sum_{i=1}^{n} (y_i-\sum_{j=1}^{p} x_{ij} \cdot \beta_j)^2 +P_{\alpha}(\beta)$$

For example consider the Elastic Net:

$$P_{\alpha}(\beta) = \alpha \cdot (\lambda\cdot \sum |\beta_j| + (1-\lambda ) \cdot \sum (\beta_j^2))$$

We can compute the partial derivatives with repect to $\beta_j$:

$$\large \frac{\partial L}{\partial \beta_j}=-2\cdot\displaystyle\sum_{i=1}^{n} (y_i-\sum_{j=1}^{p} x_{ij} \cdot \beta_j)\cdot (x_{ij}) + \alpha\cdot\lambda\cdot\text{sign}(\beta_j) + \alpha\cdot (1-\lambda)\cdot 2\cdot\beta_j$$

In sklearn you provide the L1_ratio that is $$\frac{\lambda}{1-\lambda}$$
We want to know how to update $\vec{\beta}$ so that we make the best progress in minimizing $L.$ We can think updating $\vec{\beta}$ by going $t$ units in a direction $\vec{v}$. So we ask, what is $\vec{v}$ that makes the best progress?

We consider $g(t):= L(\vec{\beta} + t \cdot \vec{v})$ SO the derivative of $g$ is

$$\large g'(t)= \nabla L \cdot \vec{v}$$ so for the most dramatic decrease of the objective we need $\vec{v}=-\nabla L$

<font color='red' size=5pt>This works well if $L$ is convex in $\beta$ and the only issue would when there is a very shallow basin of the minimum for $L.$</font>

The main goal is to approximate a ground truth coefficient vector in probability.

$$\large y = X\cdot \beta^* +\text{noise}$$

We can see the noise as $$\large \sigma\cdot \epsilon$$

So we see that:

$$\large y - X\beta^*=\sigma \cdot \epsilon$$ 

Now think about taking a partial derivative with respect to $\beta_j$ for 

$$\|y-X\beta\|_2$$

and we get 

$$- \frac{(y-X\beta)h_j(X)}{\|y-X\beta\|_2}$$

This suggests that we should rather minimize $\|y-X\beta\|_2$ + a regularization term


## Good reads about soft-thresholding:

https://www.kaggle.com/residentmario/soft-thresholding-with-lasso-regression

https://eeweb.engineering.nyu.edu/iselesni/lecture_notes/SoftThresholding.pdf



In [147]:
data = pd.read_csv('drive/MyDrive/Data Sets/concrete.csv')

In [148]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [149]:
scale = StandardScaler()
xscaled = scale.fit_transform(x)
y = y - np.mean(y)

In [150]:
def gradient_mse(beta, x, y, alpha,l): # we defined a function that computes the gradient of the objective function
    n = len(y) # the number of observations
    y_hat = x.dot(beta).flatten()
    error = (y - y_hat)
    mse = (1.0 /n) * np.sum(np.square(error)) + alpha*(l*np.sum(np.abs(beta))+(1-l)*np.sum(beta**2)) # here we have the ridge penalty
    gradient = -(2.0 /n) * error.dot(x) + 2*(1-l)*alpha*beta+alpha*l*np.sign(beta) # the penalty is baked into the gradient as well
    return gradient, mse

In [151]:
# data & hyper-parameters
alpha = 1
l = 1
x = xscaled

In [152]:
beta = np.zeros(x.shape[1])
lr = .01 # the learning rate
tolerance = 1e-8 # this would be our stopping criteria
alpha = 1
l = 1
old_w = []
mse = []

In [153]:
# Perform Gradient Descent
iterations = 1
for i in range(10000):
    gradient, mse_temp = gradient_mse(beta, xscaled, y, alpha,0.5)
    new_beta = beta - lr * gradient # here we update the coefficients in the direction of the negative gradient
    # Print error every 10 iterations
    if iterations % 1000 == 0:
        print("Iteration: %d - Mean Squarred Error: %.4f" % (iterations, mse_temp))
        old_w.append(new_beta)
        mse.append(mse_temp)
 
    # Stopping Condition
    if np.sum(abs(new_beta - beta)) < tolerance:
        print('The Gradient Descent Algorithm has converged')
        break
 
    iterations += 1
    beta = new_beta
 
print('beta =', beta)

Iteration: 1000 - Mean Squarred Error: 177.3090
The Gradient Descent Algorithm has converged
beta = [ 4.85133422  1.97685324 -0.03399281 -3.07841259  2.79580192 -1.00710962
 -1.64113454  4.04015831]


In [158]:
beta = []

In [159]:
def objective(beta):
  n = len(y) # the number of observations
  y_hat = x.dot(beta).flatten()
  error = (y - y_hat)
  mse = (1.0 /n) * np.sum(np.square(error)) + alpha*(l*np.sum(np.abs(beta))+(1-l)*np.sum(beta**2))
  return mse

In [160]:
def gradient(beta):
  n = len(y) # the number of observations
  y_hat = x.dot(beta).flatten()
  error = (y - y_hat)
  gradient = -(2.0 /n) * error.dot(x) + 2*(1-l)*alpha*beta+alpha*l*np.sign(beta)
  return gradient

In [161]:
b0 = np.zeros(x.shape[1])

In [162]:
output = minimize(objective, b0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})

In [163]:
output.x

array([ 9.15184243e+00,  5.55053209e+00,  2.26544892e+00, -3.86187813e+00,
        2.39005425e+00, -1.43391302e-07, -1.50090905e-01,  6.34592712e+00])

In [164]:
coef

array([ 9.15184243e+00,  5.55053209e+00,  2.26544892e+00, -3.86187813e+00,
        2.39005425e+00, -1.43391320e-07, -1.50090905e-01,  6.34592712e+00])

In [ ]:
coef

array([ 9.15184243e+00,  5.55053209e+00,  2.26544892e+00, -3.86187813e+00,
        2.39005425e+00, -1.43391320e-07, -1.50090905e-01,  6.34592712e+00])

In [165]:
MSE(y,xscaled.dot(coef))

110.83223565656992

## SCAD

In [166]:
def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part
    
def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

In [167]:
X = xscaled

In [168]:
def scad(beta):
  n = len(y)
  return 1/n*np.sum((y-X.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))
  
def dscad(beta):
  n = len(y)
  return (-2/n)*(y-X.dot(beta)).dot(X)+scad_derivative(beta,lam,a)

In [169]:
p = xscaled.shape[1] # p is the number of variables
b0 = [0]*p

In [54]:
(y- X.dot(b0+1)).dot(X).shape

(8,)

In [43]:
np.transpose(X).dot(y-X.dot(b0.reshape(-1,1)))

(8, 1030)

In [182]:
lam = 0.1
a = 2
output = minimize(scad, b0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})

In [183]:
output.x

array([12.26373913,  8.71309482,  5.42239662, -3.54868416,  1.61798777,
        1.1580959 ,  1.33759283,  7.21553041])

In [185]:
coef = output.x
MSE(y,xscaled.dot(coef))

107.21468778869392